# 01 · Data Acquisition & Initial Checks
Place CSVs in `../data/raw/`. This notebook loads, inspects, and writes cleaned copies to `../data/processed/`.

In [1]:
import pandas as pd
from pathlib import Path
import warnings

In [2]:
# --- Cell 1: Setup ---
# Suppress DtypeWarning as we will handle types explicitly
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

In [ ]:
# Define file paths and names
RAW = Path('../data/raw')
PRO = Path('../data/processed')
PRO.mkdir(parents=True, exist_ok=True)

PLUTO_FILE = 'pluto_25v2_1.csv' 
ACRIS_FILE = 'ACRIS_-_Real_Property_Legals_20250915.csv'
BIZ_REGISTRY_FILE = 'business_registry.csv'  # Fixed: lowercase filename
VACANT_STORE_FILE = 'Storefronts_Reported_Vacant_or_Not_20250915.csv'
DOB_PERMITS_FILE = 'DOB_Permit_Issuance_20250915.csv'
MTA_FILE = 'MTA_Subway_Hourly_Ridership__2020-2024.csv'

In [4]:
# --- Cell 2: Define Loading Functions and Data Types ---
def load_and_display(name, dtypes=None, chunksize=None):
    """Loads a CSV, optionally with specific dtypes or in chunks."""
    fp = RAW / name
    if not fp.exists():
        print(f'[ERROR] Missing file: {name}. Please place it in the `{RAW}` directory.')
        return None
    
    try:
        df_or_iterator = pd.read_csv(fp, dtype=dtypes, chunksize=chunksize)
        
        if chunksize:
            print(f'Loaded {name} as a chunk iterator.')
            return df_or_iterator
            
        print(f'Loaded {name}: {df_or_iterator.shape}')
        print("First 5 rows:")
        print(df_or_iterator.head())
        return df_or_iterator
    except Exception as e:
        print(f'[ERROR] Could not load {name}. Reason: {e}')
        return None

In [5]:
# Define dtypes for PLUTO to optimize memory
pluto_dtypes = {
    'BBL': str, 'BoroCode': 'Int8', 'Block': 'Int32', 'Lot': 'Int16', 'CD': 'Int16',
    'ZipCode': 'Int32', 'LandUse': 'category', 'OwnerType': 'category', 'BldgClass': 'category',
    'NumBldgs': 'Int16', 'NumFloors': 'float32', 'UnitsRes': 'Int16', 'UnitsTotal': 'Int16',
    'YearBuilt': 'Int16', 'YearAlter1': 'Int16', 'BuiltFAR': 'float32', 'AssessLand': 'float64',
    'AssessTot': 'float64', 'ExemptTot': 'float64', 'LotArea': 'Int64', 'BldgArea': 'Int64',
    'ComArea': 'Int64', 'ResArea': 'Int64', 'OfficeArea': 'Int64'
}

In [6]:
# --- Cell 3: Load Standard Datasets ---
print("--- Loading Standard Datasets ---")
df_pluto = load_and_display(PLUTO_FILE, dtypes=pluto_dtypes)
df_dob = load_and_display(DOB_PERMITS_FILE)
print("-" * 30)


--- Loading Standard Datasets ---
Loaded pluto_25v2_1.csv: (857736, 92)
First 5 rows:
  borough  block  lot     cd    bct2020     bctcb2020  ct2010  cb2010  \
0      QN   6421   38  407.0  4083700.0  4.083700e+10   837.0  3005.0   
1      QN   5175   20  407.0  4083700.0  4.083700e+10   837.0  3000.0   
2      QN   5173  111  407.0  4083700.0  4.083700e+10   837.0  1001.0   
3      QN   5174   16  407.0  4083700.0  4.083700e+10   837.0  3001.0   
4      QN   6413   46  407.0  4083700.0  4.083700e+10   837.0  3004.0   

   schooldist  council  ...  appbbl appdate  plutomapid  firm07_flag  \
0        25.0     20.0  ...     NaN     NaN           1          NaN   
1        25.0     20.0  ...     NaN     NaN           1          NaN   
2        25.0     20.0  ...     NaN     NaN           1          NaN   
3        25.0     20.0  ...     NaN     NaN           1          NaN   
4        25.0     20.0  ...     NaN     NaN           1          NaN   

   pfirm15_flag  version  dcpedited   lati

In [8]:
# --- Cell 4: Load and Process Large MTA Dataset in Chunks ---
print("\n--- Processing Large MTA Dataset ---")
mta_iterator = load_and_display(MTA_FILE, chunksize=500000)

if mta_iterator:
    processed_chunks = []
    for i, chunk in enumerate(mta_iterator):
        print(f'Processing MTA chunk {i+1}...')
        chunk['transit_timestamp'] = pd.to_datetime(chunk['transit_timestamp'])
        chunk = chunk[['transit_timestamp', 'station_complex_id', 'ridership']]
        # Fix: Only sum the ridership column, not the datetime column
        daily_chunk = chunk.groupby([chunk['transit_timestamp'].dt.date, 'station_complex_id'])['ridership'].sum().reset_index()
        daily_chunk.columns = ['date', 'station_complex_id', 'ridership']
        processed_chunks.append(daily_chunk)
    
    print('Concatenating processed MTA chunks...')
    df_mta_daily = pd.concat(processed_chunks).groupby(['date', 'station_complex_id']).sum().reset_index()
    print(f'Finished processing MTA data. Final shape: {df_mta_daily.shape}')
else:
    df_mta_daily = None
print("-" * 30)


--- Processing Large MTA Dataset ---
Loaded MTA_Subway_Hourly_Ridership__2020-2024.csv as a chunk iterator.
Processing MTA chunk 1...
Processing MTA chunk 1...


C:\Users\pcric\AppData\Local\Temp\ipykernel_38132\2888773040.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['transit_timestamp'] = pd.to_datetime(chunk['transit_timestamp'])


Processing MTA chunk 2...


C:\Users\pcric\AppData\Local\Temp\ipykernel_38132\2888773040.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['transit_timestamp'] = pd.to_datetime(chunk['transit_timestamp'])


Processing MTA chunk 3...


...

Processing MTA chunk 218...
Processing MTA chunk 219...
Processing MTA chunk 219...


C:\Users\pcric\AppData\Local\Temp\ipykernel_38132\2888773040.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['transit_timestamp'] = pd.to_datetime(chunk['transit_timestamp'])


Processing MTA chunk 220...


C:\Users\pcric\AppData\Local\Temp\ipykernel_38132\2888773040.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['transit_timestamp'] = pd.to_datetime(chunk['transit_timestamp'])


Processing MTA chunk 221...
Processing MTA chunk 222...
Concatenating processed MTA chunks...
Processing MTA chunk 222...
Concatenating processed MTA chunks...
Finished processing MTA data. Final shape: (769148, 3)
------------------------------
Finished processing MTA data. Final shape: (769148, 3)
------------------------------


In [10]:
# --- Cell 5: Load and Process Large ACRIS Dataset in Chunks ---
print("\n--- Processing Large ACRIS Dataset ---")
acris_iterator = load_and_display(ACRIS_FILE, chunksize=500000)

if acris_iterator:
    processed_chunks = []
    for i, chunk in enumerate(acris_iterator):
        print(f'Processing ACRIS chunk {i+1}...')
        # Convert BOROUGH, BLOCK, LOT to create BBL (Borough-Block-Lot)
        chunk['BBL'] = chunk['BOROUGH'].astype(str) + chunk['BLOCK'].astype(str).str.zfill(5) + chunk['LOT'].astype(str).str.zfill(4)
        # Use GOOD THROUGH DATE as the date field
        chunk['GOOD THROUGH DATE'] = pd.to_datetime(chunk['GOOD THROUGH DATE'], errors='coerce')
        # Select relevant columns (no DOCUMENT AMOUNT in this dataset)
        chunk = chunk[['BBL', 'DOCUMENT ID', 'GOOD THROUGH DATE']].dropna(subset=['BBL', 'GOOD THROUGH DATE'])
        processed_chunks.append(chunk)

    print('Concatenating processed ACRIS chunks...')
    df_acris_full = pd.concat(processed_chunks)
    
    print('Aggregating ACRIS data by BBL...')
    # Sort by date to find the most recent record
    df_acris_full = df_acris_full.sort_values('GOOD THROUGH DATE', ascending=False)
    
    # Aggregate to get last record info and record counts
    acris_agg = df_acris_full.groupby('BBL').agg(
        last_good_through_date=('GOOD THROUGH DATE', 'first'),
        last_document_id=('DOCUMENT ID', 'first'),
        record_count=('BBL', 'size')
    ).reset_index()
    
    df_acris = acris_agg
    print(f'Finished processing ACRIS data. Final shape: {df_acris.shape}')
else:
    df_acris = None
print("-" * 30)


--- Processing Large ACRIS Dataset ---
Loaded ACRIS_-_Real_Property_Legals_20250915.csv as a chunk iterator.
Processing ACRIS chunk 1...
Processing ACRIS chunk 1...
Processing ACRIS chunk 2...
Processing ACRIS chunk 2...
Processing ACRIS chunk 3...
Processing ACRIS chunk 3...
Processing ACRIS chunk 4...
Processing ACRIS chunk 4...
Processing ACRIS chunk 5...
Processing ACRIS chunk 5...
Processing ACRIS chunk 6...
Processing ACRIS chunk 6...
Processing ACRIS chunk 7...
Processing ACRIS chunk 7...
Processing ACRIS chunk 8...
Processing ACRIS chunk 8...
Processing ACRIS chunk 9...
Processing ACRIS chunk 9...
Processing ACRIS chunk 10...
Processing ACRIS chunk 10...
Processing ACRIS chunk 11...
Processing ACRIS chunk 11...
Processing ACRIS chunk 12...
Processing ACRIS chunk 12...
Processing ACRIS chunk 13...
Processing ACRIS chunk 13...
Processing ACRIS chunk 14...
Processing ACRIS chunk 14...
Processing ACRIS chunk 15...
Processing ACRIS chunk 15...
Processing ACRIS chunk 16...
Processin

In [16]:
# --- Cell 6: Standardize Columns and Save Processed Files ---
print("\n--- Saving Processed Files ---")
def to_snake_case(s):
    """Converts a string to snake_case."""
    return s.strip().lower().replace(' ', '_').replace('-', '_').replace('.', '')

datasets = {
    'pluto': df_pluto,
    'dob_permits': df_dob,
    'mta_daily_ridership': df_mta_daily,
    'acris_aggregated': df_acris,
    'business_registry': df_business,
    'vacant_storefronts': df_vacant
}

for name, df in datasets.items():
    if df is None:
        print(f'Skipping {name} as it was not loaded.')
        continue
        
    df.columns = [to_snake_case(c) for c in df.columns]
    
    if 'bbl' in df.columns:
        df['bbl'] = df['bbl'].astype(str).str.split('.').str[0]
    
    output_path = PRO / f"{name}_processed.csv"
    df.to_csv(output_path, index=False)
    print(f'Successfully saved processed file to: {output_path}')
print("-" * 30)




--- Saving Processed Files ---
Successfully saved processed file to: ..\data\processed\pluto_processed.csv
Successfully saved processed file to: ..\data\processed\pluto_processed.csv
Successfully saved processed file to: ..\data\processed\dob_permits_processed.csv
Successfully saved processed file to: ..\data\processed\dob_permits_processed.csv
Successfully saved processed file to: ..\data\processed\mta_daily_ridership_processed.csv
Successfully saved processed file to: ..\data\processed\mta_daily_ridership_processed.csv
Successfully saved processed file to: ..\data\processed\acris_aggregated_processed.csv
Successfully saved processed file to: ..\data\processed\acris_aggregated_processed.csv
Successfully saved processed file to: ..\data\processed\business_registry_processed.csv
Successfully saved processed file to: ..\data\processed\business_registry_processed.csv
Successfully saved processed file to: ..\data\processed\vacant_storefronts_processed.csv
------------------------------
Su

In [12]:
# --- Cell 7: Load Business Registry Dataset ---
print("\n--- Loading Business Registry Dataset ---")
df_business = load_and_display(BIZ_REGISTRY_FILE)
print("-" * 30)


--- Loading Business Registry Dataset ---
Loaded Business_Registry.csv: (66425, 31)
First 5 rows:
  License Number                Business Name DBA/Trade Name  \
0   2123226-DCWP             Denis Spedalieri            NaN   
1    2089275-DCA  SANJAY'S VARIETY STORE INC.            NaN   
2   2123284-DCWP                 Gayla Hibner            NaN   
3    2080184-DCA    FAMILY CARE REFERRAL, LLC            NaN   
4   2123026-DCWP                   Donna Hill            NaN   

  Business Unique ID            Business Category License Type License Status  \
0    BA-1703334-2024            Sightseeing Guide   Individual         Active   
1    BA-1500311-2022  Secondhand Dealer - General     Premises        Expired   
2    BA-1698945-2024            Sightseeing Guide   Individual         Active   
3    BA-1269913-2022            Employment Agency     Premises         Active   
4    BA-1696202-2024            Sightseeing Guide   Individual         Active   

  Initial Issuance Date Expir

In [15]:
# --- Cell 8: Load Vacant Storefront Dataset ---
print("\n--- Loading Vacant Storefront Dataset ---")
df_vacant = load_and_display(VACANT_STORE_FILE)
print("-" * 30)

# --- Cell 9: Add Missing Datasets to Processing ---
print("\n--- Adding Missing Datasets to Processing ---")
# Add the new datasets to the existing datasets dictionary
datasets['business_registry'] = df_business
datasets['vacant_storefronts'] = df_vacant
print("Added business_registry and vacant_storefronts to processing queue")
print("-" * 30)


--- Loading Vacant Storefront Dataset ---
Loaded Storefronts_Reported_Vacant_or_Not_20250915.csv: (348297, 27)
First 5 rows:
  Filing Due Date Reporting Year  Borough Block Lot  \
0      06/03/2024           2023         5051430020   
1      06/03/2024           2023         5005430010   
2      06/03/2024           2023         5051430017   
3      06/03/2024           2023         5006550014   
4      06/03/2024           2023         5001730034   

  Property Street Address or Storefront Address        Borough Zip Code  \
0                               3996 AMBOY ROAD  STATEN ISLAND    10308   
1                              271 BROAD STREET  STATEN ISLAND    10304   
2                               3 NELSON AVENUE  STATEN ISLAND    10308   
3                               1366 CLOVE ROAD  STATEN ISLAND    10301   
4                          693 HENDERSON AVENUE  STATEN ISLAND    10310   

  Sold Date Vacant on 12/31 Construction Reported Vacant 6/30 or Date Sold  \
0       NaN   